In [2]:
from sympy import *

<font size= 6> Specific case: a flat free surface: </font>
1. `dVxdy = -dVydx`
2. `dVydy = 1/2*dVxdx + 3/4*P/eta`

The Fictitious values are:
1. `VxN = −dVydx*dy + VxS`
2. `VyN = 0.5*dVxdx*dy + 0.75*dz*P/eta+vS`

see notebook: FreeSurface_v3.ipynb

<font size= 6> Discretisation of the linear momentum balance </font>

In [3]:
ani = 1

In [4]:
def d_dx1(uE,uW,uN,uS,dksi,deta,dksidx,detadx):
    return dksidx*(uE-uW)/dksi + detadx*(uN-uS)/deta
def d_dy1(vE,vW,vN,vS,dksi,deta,dksidy,detady):
    return dksidy*(vE-vW)/dksi + detady*(vN-vS)/deta

In [5]:
dx, dy, dt = symbols('dx, dy, dt')
VxC, VxW, VxE, VxS, VxN = symbols('VxC, VxW, VxE, VxS, VxN')
VyC, VyW, VyE, VyS, VyN = symbols('VyC, VyW, VyE, VyS, VyN')
VySW, VySE, VyNW, VyNE = symbols('VySW, VySE, VyNW, VyNE')
VxSW, VxSE, VxNW, VxNE = symbols('VxSW, VxSE, VxNW, VxNE')
pS, pN, pE, pW = symbols('pS, pN, pE, pW')
D11E, D11W, D12E, D12W, D13E, D13W = symbols('D11E, D11W, D12E, D12W, D13E, D13W')
D21N, D21S, D22N, D22S, D23N, D23S = symbols('D21N, D21S, D22N, D22S, D23N, D23S')
D11N, D11S, D12N, D12S, D13N, D13S = symbols('D11N, D11S, D12N, D12S, D13N, D13S')
D21E, D21W, D22E, D22W, D23E, D23W = symbols('D21E, D21W, D22E, D22W, D23E, D23W')
D31E, D31W, D32E, D32W, D33E, D33W = symbols('D31E, D31W, D32E, D32W, D33E, D33W')
D31N, D31S, D32N, D32S, D33S, D33N = symbols('D31N, D31S, D32N, D32S, D33S, D33N')

In [6]:
dofsV = [VxC, VxW, VxE, VxS, VxN, VxSW, VxSE, VxNW, VxNE,
         VyC, VyW, VyE, VyS, VyN, VySW, VySE, VyNW, VyNE]
dofsP = [ pW, pE, pS, pN ]

In [7]:
aW,bW,cW,dW = symbols('aW,bW,cW,dW')
aE,bE,cE,dE = symbols('aE,bE,cE,dE')
aS,bS,cS,dS = symbols('aS,bS,cS,dS')
aN,bN,cN,dN = symbols('aN,bN,cN,dN')
aC,bC,cC,dC = symbols('aC,bC,cC,dC')

In [8]:
# Velocity gradients
# 1. `dVxdy = -dVydx`
# 2. `dVydy = 1/2*dVxdx + 3/4*P/eta`
P_surf, eta_surf = symbols('P_surf, eta_surf')
dVxdxE = d_dx1(VxE,VxC,VxNE,VxSE,dx,dy,aE,bE)
dVydyE = d_dy1(VyE,VyC,VyNE,VySE,dx,dy,cE,dE)
dVxdxW = d_dx1(VxC,VxW,VxNW,VxSW,dx,dy,aW,bW)
dVydyW = d_dy1(VyC,VyW,VyNW,VySW,dx,dy,cW,dW)
# ----------
dVxdxN = d_dx1(VxNE,VxNW,VxN,VxC,dx,dy,aN,bN)
# dVydyN = d_dy1(VyNE,VyNW,VyN,VyC,dx,dy,cN,dN)
dVydyN  = Rational(1,2)*dVxdxN + Rational(3,4)*P_surf/eta_surf
dVxdxS = d_dx1(VxSE,VxSW,VxC,VxS,dx,dy,aS,bS)
dVydyS = d_dy1(VySE,VySW,VyC,VyS,dx,dy,cS,dS)
# ----------
dVydxN = d_dx1(VyNE,VyNW,VyN,VyC,dx,dy,aN,bN)
# dVxdyN = d_dy1(VxNE,VxNW,VxN,VxC,dx,dy,cN,dN)
dVxdyN = -dVydxN
dVydxS = d_dx1(VySE,VySW,VyC,VyS,dx,dy,aS,bS)
dVxdyS = d_dy1(VxSE,VxSW,VxC,VxS,dx,dy,cS,dS)
# ----------
dVydxE = d_dx1(VyE,VyC,VyNE,VySE,dx,dy,aE,bE)
dVxdyE = d_dy1(VxE,VxC,VxNE,VxSE,dx,dy,cE,dE)
dVydxW = d_dx1(VyC,VyW,VyNW,VySW,dx,dy,aW,bW)
dVxdyW = d_dy1(VxC,VxW,VxNW,VxSW,dx,dy,cW,dW)

In [9]:
# Kinematics
divE   = dVxdxE + dVydyE
divW   = dVxdxW + dVydyW
divN   = dVxdxN + dVydyN
divS   = dVxdxS + dVydyS
ExxE   = dVxdxE - Rational(1,3)*divE
ExxW   = dVxdxW - Rational(1,3)*divW
ExxN   = dVxdxN - Rational(1,3)*divN
ExxS   = dVxdxS - Rational(1,3)*divS
EyyE   = dVydyE - Rational(1,3)*divE
EyyW   = dVydyW - Rational(1,3)*divW
EyyN   = dVydyN - Rational(1,3)*divN
EyyS   = dVydyS - Rational(1,3)*divS
ExyN   = Rational(1,2)*(dVxdyN + dVydxN)
ExyS   = Rational(1,2)*(dVxdyS + dVydxS)
ExyE   = Rational(1,2)*(dVxdyE + dVydxE)
ExyW   = Rational(1,2)*(dVxdyW + dVydxW)

In [10]:
# Deviatoric stress
TxxE   =     D11E*ExxE + ani*D12E*EyyE + ani*D13E*ExyE
TxxW   =     D11W*ExxW + ani*D12W*EyyW + ani*D13W*ExyW
TxxN   =     D11N*ExxN + ani*D12N*EyyN + ani*D13N*ExyN
TxxS   =     D11S*ExxS + ani*D12S*EyyS + ani*D13S*ExyS
TyyE   = ani*D21E*ExxE +     D22E*EyyE + ani*D23E*ExyE
TyyW   = ani*D21W*ExxW +     D22W*EyyW + ani*D23W*ExyW
TyyN   = ani*D21N*ExxN +     D22N*EyyN + ani*D23N*ExyN
TyyS   = ani*D21S*ExxS +     D22S*EyyS + ani*D23S*ExyS
TxyE   = ani*D31E*ExxE + ani*D32E*EyyE +     D33E*ExyE
TxyW   = ani*D31W*ExxW + ani*D32W*EyyW +     D33W*ExyW
TxyN   = ani*D31N*ExxN + ani*D32N*EyyN +     D33N*ExyN
TxyS   = ani*D31S*ExxS + ani*D32S*EyyS +     D33S*ExyS

In [11]:
# Linear momentum balance
Fx = -( d_dx1(TxxE,TxxW,TxxN,TxxS,dx,dy,aC,bC) + d_dy1(TxyE,TxyW,TxyN,TxyS,dx,dy,cC,dC) + d_dx1(pE,pW,pN,pS,dx,dy,aC,bC) )
Fy = -( d_dy1(TyyE,TyyW,TyyN,TyyS,dx,dy,cC,dC) + d_dx1(TxyE,TxyW,TxyN,TxyS,dx,dy,aC,bC) + d_dy1(pE,pW,pN,pS,dx,dy,cC,dC) )

In [12]:
a = ('D11E', 'D11W', 'D12E', 'D12W', 'D13E', 'D13W',
     'D21N', 'D21S', 'D22N', 'D22S', 'D23N', 'D23S',
     'D11N', 'D11S', 'D12N', 'D12S', 'D13N', 'D13S',
     'D21E', 'D21W', 'D22E', 'D22W', 'D23E', 'D23W',
     'D31E', 'D31W', 'D32E', 'D32W', 'D33E', 'D33W',
     'D31N', 'D31S', 'D32N', 'D32S', 'D33S', 'D33N')
b = ('D.D11E', 'D.D11W', 'D.D12E', 'D.D12W', 'D.D13E', 'D.D13W',
     'D.D21N', 'D.D21S', 'D.D22N', 'D.D22S', 'D.D23N', 'D.D23S',
     'D.D11N', 'D.D11S', 'D.D12N', 'D.D12S', 'D.D13N', 'D.D13S',
     'D.D21E', 'D.D21W', 'D.D22E', 'D.D22W', 'D.D23E', 'D.D23W',
     'D.D31E', 'D.D31W', 'D.D32E', 'D.D32W', 'D.D33E', 'D.D33W',
     'D.D31N', 'D.D31S', 'D.D32N', 'D.D32S', 'D.D33S', 'D.D33N')
c = ('aW','bW','cW','dW',
     'aE','bE','cE','dE',
     'aS','bS','cS','dS',
     'aN','bN','cN','dN',
     'aC','bC','cC','dC')
d = ('a.W','b.W','c.W','d.W',
     'a.E','b.E','c.E','d.E',
     'a.S','b.S','c.S','d.S',
     'a.N','b.N','c.N','d.N',
     'a.C','b.C','c.C','d.C')

In [13]:
i=0
for dof in dofsV:
    i+=1
    coeff = Fx.diff(dof)
    final = 'v_uu[' + str(i) + '] = ' + julia_code(coeff.simplify())
    for id in range(36):
        final = final.replace( a[id], b[id] )
    for id in range(20):
        final = final.replace( c[id], d[id] )
    print(final)
i=0
for dof in dofsP:
    i+=1
    coeff = Fx.diff(dof)
    print('v_up[' + str(i) + '] = ' + julia_code(coeff.simplify()))

v_uu[1] = (dx .^ 2 .* (b.C .* (3 * D.D11N .* b.N + 4 * D.D11S .* b.S - 2 * D.D12S .* b.S + 3 * D.D13S .* d.S) + d.C .* (3 * D.D31N .* b.N + 4 * D.D31S .* b.S - 2 * D.D32S .* b.S + 3 * D.D33S .* d.S)) + dy .^ 2 .* (a.C .* (4 * D.D11E .* a.E + 4 * D.D11W .* a.W - 2 * D.D12E .* a.E - 2 * D.D12W .* a.W + 3 * D.D13E .* c.E + 3 * D.D13W .* c.W) + c.C .* (4 * D.D31E .* a.E + 4 * D.D31W .* a.W - 2 * D.D32E .* a.E - 2 * D.D32W .* a.W + 3 * D.D33E .* c.E + 3 * D.D33W .* c.W))) ./ (6 * dx .^ 2 .* dy .^ 2)
v_uu[2] = (-a.C .* (4 * D.D11W .* a.W - 2 * D.D12W .* a.W + 3 * D.D13W .* c.W) - c.C .* (4 * D.D31W .* a.W - 2 * D.D32W .* a.W + 3 * D.D33W .* c.W)) ./ (6 * dx .^ 2)
v_uu[3] = (-a.C .* (4 * D.D11E .* a.E - 2 * D.D12E .* a.E + 3 * D.D13E .* c.E) - c.C .* (4 * D.D31E .* a.E - 2 * D.D32E .* a.E + 3 * D.D33E .* c.E)) ./ (6 * dx .^ 2)
v_uu[4] = (-b.C .* (4 * D.D11S .* b.S - 2 * D.D12S .* b.S + 3 * D.D13S .* d.S) - d.C .* (4 * D.D31S .* b.S - 2 * D.D32S .* b.S + 3 * D.D33S .* d.S)) ./ (6 * dy .^ 2)
v_

In [14]:
i=0
for dof in dofsV:
    i+=1
    coeff = Fy.diff(dof)
    # print('c' + str(dof) + ' = ' + julia_code(coeff.simplify()))
    final = 'v_uu[' + str(i) + '] = ' + julia_code(coeff.simplify())
    for id in range(36):
        final = final.replace( a[id], b[id] )
    for id in range(20):
        final = final.replace( c[id], d[id] )
    print(final)
i=0
for dof in dofsP:
    i+=1
    coeff = Fy.diff(dof)
    print('v_up[' + str(i) + '] = ' + julia_code(coeff.simplify()))

v_uu[1] = (dx .^ 2 .* (b.C .* (3 * D.D31N .* b.N + 4 * D.D31S .* b.S - 2 * D.D32S .* b.S + 3 * D.D33S .* d.S) + d.C .* (3 * D.D21N .* b.N + 4 * D.D21S .* b.S - 2 * D.D22S .* b.S + 3 * D.D23S .* d.S)) + dy .^ 2 .* (a.C .* (4 * D.D31E .* a.E + 4 * D.D31W .* a.W - 2 * D.D32E .* a.E - 2 * D.D32W .* a.W + 3 * D.D33E .* c.E + 3 * D.D33W .* c.W) + c.C .* (4 * D.D21E .* a.E + 4 * D.D21W .* a.W - 2 * D.D22E .* a.E - 2 * D.D22W .* a.W + 3 * D.D23E .* c.E + 3 * D.D23W .* c.W))) ./ (6 * dx .^ 2 .* dy .^ 2)
v_uu[2] = (-a.C .* (4 * D.D31W .* a.W - 2 * D.D32W .* a.W + 3 * D.D33W .* c.W) - c.C .* (4 * D.D21W .* a.W - 2 * D.D22W .* a.W + 3 * D.D23W .* c.W)) ./ (6 * dx .^ 2)
v_uu[3] = (-a.C .* (4 * D.D31E .* a.E - 2 * D.D32E .* a.E + 3 * D.D33E .* c.E) - c.C .* (4 * D.D21E .* a.E - 2 * D.D22E .* a.E + 3 * D.D23E .* c.E)) ./ (6 * dx .^ 2)
v_uu[4] = (-b.C .* (4 * D.D31S .* b.S - 2 * D.D32S .* b.S + 3 * D.D33S .* d.S) - d.C .* (4 * D.D21S .* b.S - 2 * D.D22S .* b.S + 3 * D.D23S .* d.S)) ./ (6 * dy .^ 2)
v_

<font size= 6> Discretisation of the continuity equation </font>

In [15]:
VxE,VxW,VyN,VyS = symbols('VxE,VxW,VyN,VyS')
VyE,VyW,VxN,VxS = symbols('VyE,VyW,VxN,VxS')
PC, PC0, K = symbols('PC, PC_0, K')
aC,bC,cC,dC = symbols('aC,bC,cC,dC') 

In [16]:
dofs_pu = [VxW,VxE,VxS,VxN,VyW,VyE,VyS,VyN,PC]

In [17]:
divC = d_dx1(VxE,VxW,VxN,VxS,dx,dy,aC,bC) + d_dy1(VyE,VyW,VyN,VyS,dx,dy,cC,dC)
Fp   = -divC + (PC-PC0)/(K*dt)

In [18]:
i=0
for dof in dofs_pu:
    i+=1
    coeff = Fp.diff(dof)
    print('v_pu[' + str(i) + '] = ' + julia_code(coeff.simplify()))

v_pu[1] = aC ./ dx
v_pu[2] = -aC ./ dx
v_pu[3] = bC ./ dy
v_pu[4] = -bC ./ dy
v_pu[5] = cC ./ dx
v_pu[6] = -cC ./ dx
v_pu[7] = dC ./ dy
v_pu[8] = -dC ./ dy
v_pu[9] = 1 ./ (K .* dt)
